In [1]:
import mediapipe as mp
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


### Load Model

In [2]:
import tensorflow as tf
# Memuat model dari file .keras
model = tf.keras.models.load_model('model3.keras')

# Verifikasi arsitektur model
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_9 (Flatten)                  │ (None, 126)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 512)                 │          65,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_36 (Dense)                     │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 26)                  │             858 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 702,704 (2.68 MB)

 Trainable params: 234,234 (914.98 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 468,470 (1.79 MB)

In [7]:
class input_pipeline:
    def __init__(self, task = 'hand_landmarker.task', model = 'model1.keras'):
        #Inisialisasi Model Mediapipe
        base_options = python.BaseOptions(model_asset_path= task)
        options = vision.HandLandmarkerOptions(base_options=base_options,
                                               num_hands=2)
        detector = vision.HandLandmarker.create_from_options(options)

        #Inisialisasi Model machinelearning
        model = tf.keras.models.load_model(model)

        self.detector = detector
        self.model = model

    def __call__(self, frame):
        self.extracted_landmarks = self.end2endProcess(frame)
        return self.extracted_landmarks
    
    def predict_landmarks(self, frame):
        image = mp.Image(image_format = mp.ImageFormat.SRGB, data = frame)

        return self.detector.detect(image)

    
    def adjust_landmarks(self, hand_landmarks):
        reference_point = hand_landmarks[0]
        adjusted_landmarks = []
        
        for landmark in hand_landmarks:
            adjusted_landmarks.append([
                landmark[0] - reference_point[0],
                landmark[1] - reference_point[1],
                landmark[2] - reference_point[2]
            ])
        return np.array(adjusted_landmarks)

    
    def extract_keypoints(self, landmark_result):
        lh = np.zeros((21,3))
        rh = np.zeros((21,3))
        
        if landmark_result.handedness is not None:
        #Ekstrak koordinat landmarks
            for index, handedness in enumerate(landmark_result.handedness):
                if handedness[0].display_name == 'Right':
                    rh = [[landmark.x, landmark.y, landmark.z]for landmark in landmark_result.hand_landmarks[index]]
                if handedness[0].display_name == 'Left':
                    lh = [[landmark.x, landmark.y, landmark.z]for landmark in landmark_result.hand_landmarks[index]]

        #Gabungkan Landmarks
        self.lh = self.adjust_landmarks(lh)
        self.rh = self.adjust_landmarks(rh)
        self.concatenated_landmarks = np.concatenate([
            self.lh.flatten(),
            self.rh.flatten()
        ])
        
        #Gabungkan landmarks menjadi satu list
        return self.concatenated_landmarks


    def end2endProcess(self, frame):
        mediapipe_result = self.predict_landmarks(frame) 
        extracted_keypoints = self.extract_keypoints(mediapipe_result)

        return extracted_keypoints

    def predict_sign(self, treshold = 0.5):
        expanded_landmarks = np.expand_dims(self.extracted_landmarks, axis = 0)
        self.prediksi = model.predict(expanded_landmarks)
        predicted_class = np.where(self.prediksi[0] >= treshold )[0] if np.where(self.prediksi[0] >= treshold )[0].size > 0 else None
        
        return predicted_class, self.prediksi

In [10]:
testClass = input_pipeline()

In [11]:
testClass(cv2.cvtColor(cv2.imread("D:\Kuliah\StudiIndependen-Bangkit-ML-2024\capstone\data_Nanta\G\IMG20230207085011_jpg.rf.8c1a849e9b5795a589ebb3f03f625bb5.jpg"), cv2.COLOR_BGR2RGB))

array([ 0.        ,  0.        ,  0.        , -0.02856439, -0.0632028 ,
        0.00508087, -0.09323025, -0.09631053, -0.00206872, -0.15724909,
       -0.09975663, -0.00876094, -0.19729316, -0.08600384, -0.01571309,
       -0.08294946, -0.08006623, -0.02804998, -0.16432822, -0.09181619,
       -0.03749431, -0.16208029, -0.08442396, -0.03917675, -0.14904648,
       -0.07703522, -0.03894551, -0.08873308, -0.03897509, -0.03502393,
       -0.17451382, -0.05646074, -0.03830078, -0.16431409, -0.05714068,
       -0.03270792, -0.14420265, -0.05534264, -0.03146545, -0.09598911,
       -0.0023202 , -0.03908794, -0.17055547, -0.02626711, -0.03977313,
       -0.15910417, -0.03149414, -0.02814142, -0.13983011, -0.0315837 ,
       -0.02278631, -0.10334456,  0.02840099, -0.04183211, -0.15557259,
        0.00457636, -0.03974959, -0.14598602, -0.00218058, -0.02810943,
       -0.13153809, -0.00155562, -0.02144485,  0.        ,  0.        ,
        0.        ,  0.05240029, -0.04680064,  0.00515131,  0.09

In [12]:
hasil, array_hasil = testClass.predict_sign()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


In [13]:
valid_indices = np.where(array_hasil[0] >= 1)[0] if np.where(array_hasil[0] >= 0.5)[0].size > 0 else None
print(valid_indices)

[]


In [14]:
print(hasil)

[6]


In [16]:
# Data array
data = np.array([[2.7437562e-36, 2.2299391e-17, 4.5922667e-27, 0.0000000e+00,
                  7.3517858e-12, 2.0128831e-11, 1.0000000e+00, 0.0000000e+00,
                  0.0000000e+00, 1.9008328e-15, 0.0000000e+00, 2.8198281e-35,
                  8.3032325e-32, 6.8006050e-22, 8.2360395e-32, 5.3941737e-35,
                  1.2566813e-29, 0.0000000e+00, 2.5070255e-17, 0.0000000e+00,
                  6.8833360e-16, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
                  3.9816983e-11, 4.6761695e-10]], dtype=np.float32)

# Threshold
threshold = 

# Ternary condition to check and assign values
result = data[0][data[0] >= threshold] if np.any(data[0] >= threshold) else 0

# Output
print("Result:", result)


SyntaxError: invalid syntax (3427562644.py, line 11)